In [1]:
import MDAnalysis as mda
import pandas as pd
from biopandas.pdb import PandasPdb
import os
import glob
import re
import math
import numpy as np
from rdkit import Chem
from scipy.spatial.transform import Rotation as R

def grid_list(atom_df):
    return list(zip(atom_df['x_coord'], atom_df['y_coord'], atom_df['z_coord']))

def filtering_proteins(atom_df, grid_list, radius=5.0):
    atom_coords = atom_df[['x_coord', 'y_coord', 'z_coord']].values
    filtered_atoms = set()

    for x, y, z in grid_list:
        distances_sq = (atom_coords[:, 0] - x)**2 + (atom_coords[:, 1] - y)**2 + (atom_coords[:, 2] - z)**2
        mask = distances_sq <= radius**2
        filtered_atoms.update(atom_df.index[mask])

    print(f"Total atoms within {radius} Å cutoff: {len(filtered_atoms)}")
    return atom_df.loc[list(filtered_atoms)]


In [2]:
def get_positive_ligand_atoms(positive_file, protein_name):
    protein_pdb_df = PandasPdb().read_pdb(positive_file)
    protein_pdb_df.df.keys()
    protein = protein_pdb_df.df['ATOM']
    protein = protein[~protein['atom_name'].str.startswith('H')] # don't use hydrogen
    protein_coords = protein[['x_coord', 'y_coord', 'z_coord']].values
    protein_centroid = protein_coords.mean(axis=0)
    print(set(protein['chain_id']))
    print(positive_file)

    ligand_df = PandasPdb().read_pdb(positive_file)
    ligand_df.df.keys()
    ligand = ligand_df.df['HETATM']
    ligand = ligand[ligand['residue_name']=="CLR"]
    x = list(set(zip(ligand['residue_number'], ligand['chain_id'])))

    #get the most inward residue
    min_distance = float('inf')
    closest_clr = None

    all_ligands = []

    for residue_number, chain_id in x:
        clr_atoms = ligand[(ligand['residue_number'] == residue_number) & (ligand['chain_id'] == chain_id)]
        if clr_atoms.empty:
            continue

        clr_coords = clr_atoms[['x_coord', 'y_coord', 'z_coord']].values
        clr_centroid = clr_coords.mean(axis=0)
        
        distance = np.linalg.norm(protein_centroid - clr_centroid)
        
        if distance < min_distance:
            min_distance = distance
            closest_clr = (residue_number, chain_id)

        grid_list_ = grid_list(clr_atoms)

        all_ligands.append(filtering_proteins(protein, grid_list_))

    ligand_ = ligand[(ligand['residue_number'] == closest_clr[0]) & (ligand['chain_id'] == closest_clr[1])]
    grid_list_ = grid_list(ligand_)

    filtered_atoms = filtering_proteins(protein, grid_list_)

    # Save to pdb
    filtered_pdb = PandasPdb()
    filtered_pdb.df['ATOM'] = filtered_atoms
    filtered_pdb_path = f"filtered-pdbs-distinct-5A/positive/{protein_name}-filtered.pdb"
    os.makedirs(os.path.dirname(filtered_pdb_path), exist_ok=True)
    filtered_pdb.to_pdb(path=filtered_pdb_path, records=None, gz=False, append_newline=True)

    return protein, all_ligands


In [3]:
def check_if_unlabeled_is_positive(positive_df, unlabeled_df):
    # Create a unique key for each atom based on identifying features
    positive_df['atom_key'] = (
        positive_df['atom_name'].str.strip() + '_' +
        positive_df['residue_name'].str.strip() + '_' +
        positive_df['residue_number'].astype(str) + '_' +
        positive_df['chain_id'].fillna('')
    )

    unlabeled_df['atom_key'] = (
        unlabeled_df['atom_name'].str.strip() + '_' +
        unlabeled_df['residue_name'].str.strip() + '_' +
        unlabeled_df['residue_number'].astype(str) + '_' +
        unlabeled_df['chain_id'].fillna('')
    )

    keys1 = set(positive_df['atom_key'])
    keys2 = set(unlabeled_df['atom_key'])

    common_atoms = keys1 & keys2
    total_atoms = max(len(keys1), len(keys2))

    if total_atoms == 0:
        print("Zero total atoms")
        return False

    similarity = len(common_atoms) / total_atoms
    print(similarity)
    return similarity >= 0.2, similarity

In [4]:
def get_protein_name(filename):
    basename = os.path.basename(filename)  # Get file name without path
    match = re.match(r'([a-zA-Z0-9]{4})', basename)  # Match the first 4-character PDB ID
    if match:
        return match.group(1).upper()
    else:
        return None
def get_mode_index(filename):
    basename = os.path.basename(filename)
    match = re.search(r'mode_(\d+)', basename)
    if match:
        return int(match.group(1))
    else:
        return None  # or raise ValueError("No mode index found.")

def natural_sort_key(s):
    """Function to sort strings in a natural alphanumeric order."""
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]


In [5]:
positive_files = glob.glob("../3DCNN/CLR-PDB/*.pdb")
positive_files = sorted(positive_files, key=natural_sort_key)

unlabeled_files = glob.glob("../3DCNN/CLR-Unlabeled-Distinct/*.pdb")
unlabeled_files = sorted(unlabeled_files, key=natural_sort_key)

positive_index = 0
protein, all_lig_filtered = get_positive_ligand_atoms(positive_files[positive_index], get_protein_name(positive_files[positive_index]))

for unlabeled_file in unlabeled_files:
    positive_name = get_protein_name(positive_files[positive_index])
    unlabeled_name = get_protein_name(unlabeled_file)

    fragment_index = get_mode_index(unlabeled_file)

    if positive_name != unlabeled_name:
        positive_index += 1
        positive_name = get_protein_name(positive_files[positive_index])

        if positive_name != unlabeled_name:
            raise Exception("Proteins Not Matching Up!!!")
        
        protein, all_lig_filtered = get_positive_ligand_atoms(positive_files[positive_index], positive_name)

    fragment_df = PandasPdb().read_pdb(unlabeled_file)
    fragment_df.df.keys()
    fragment = fragment_df.df['HETATM']

    grid_list_ = grid_list(fragment)

    filtered_atoms = filtering_proteins(protein, grid_list_)
    
    if not filtered_atoms.empty:
        for lig in all_lig_filtered:
            is_positive, similarity = check_if_unlabeled_is_positive(lig, filtered_atoms)

            if is_positive:
                break

        # Save to pdb
        filtered_pdb = PandasPdb()
        filtered_pdb.df['ATOM'] = filtered_atoms

        if is_positive:
            filtered_pdb_path = f"filtered-pdbs-distinct-5A/unlabeled/{unlabeled_name}-f{fragment_index}-positive-{similarity:0.3f}.pdb"
        else:
            filtered_pdb_path = f"filtered-pdbs-distinct-5A/unlabeled/{unlabeled_name}-f{fragment_index}-{similarity:0.3f}.pdb"
        os.makedirs(os.path.dirname(filtered_pdb_path), exist_ok=True)
        filtered_pdb.to_pdb(path=filtered_pdb_path, records=None, gz=False, append_newline=True)
    
    fragment_index += 1

{'A'}
../3DCNN/CLR-PDB/1LRI_protein.pdb
Total atoms within 5.0 Å cutoff: 71
Total atoms within 5.0 Å cutoff: 71
Total atoms within 5.0 Å cutoff: 70
0.18309859154929578
Total atoms within 5.0 Å cutoff: 45
0.16901408450704225
{'A'}
../3DCNN/CLR-PDB/1N83_protein.pdb
Total atoms within 5.0 Å cutoff: 75
Total atoms within 5.0 Å cutoff: 75


/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(


Total atoms within 5.0 Å cutoff: 75
0.0
Total atoms within 5.0 Å cutoff: 67
0.0
Total atoms within 5.0 Å cutoff: 54
0.0
{'A'}
../3DCNN/CLR-PDB/1ZHY_protein.pdb
Total atoms within 5.0 Å cutoff: 77
Total atoms within 5.0 Å cutoff: 77
Total atoms within 5.0 Å cutoff: 75
0.0


/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(


Total atoms within 5.0 Å cutoff: 78
0.0
Total atoms within 5.0 Å cutoff: 111
0.0
Total atoms within 5.0 Å cutoff: 88
0.0
Total atoms within 5.0 Å cutoff: 79
0.0
{'A'}
../3DCNN/CLR-PDB/2RH1_protein.pdb
Total atoms within 5.0 Å cutoff: 17
Total atoms within 5.0 Å cutoff: 53
Total atoms within 5.0 Å cutoff: 32
Total atoms within 5.0 Å cutoff: 53


...

/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(


{'B', 'C', 'R', 'A'}
../3DCNN/CLR-PDB/9K1D_protein.pdb
Total atoms within 5.0 Å cutoff: 38
Total atoms within 5.0 Å cutoff: 16
Total atoms within 5.0 Å cutoff: 26
Total atoms within 5.0 Å cutoff: 64
Total atoms within 5.0 Å cutoff: 23
Total atoms within 5.0 Å cutoff: 64
Total atoms within 5.0 Å cutoff: 35
0.5526315789473685
Total atoms within 5.0 Å cutoff: 39
0.3076923076923077
Total atoms within 5.0 Å cutoff: 82
0.0
0.0
0.0
0.0
0.0
Total atoms within 5.0 Å cutoff: 86
0.0
0.0
0.0
0.0
0.0


/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(


{'A'}
../3DCNN/CLR-PDB/9K7B_protein.pdb
Total atoms within 5.0 Å cutoff: 36
Total atoms within 5.0 Å cutoff: 33
Total atoms within 5.0 Å cutoff: 36
Total atoms within 5.0 Å cutoff: 49
Total atoms within 5.0 Å cutoff: 39
Total atoms within 5.0 Å cutoff: 36
Total atoms within 5.0 Å cutoff: 61
0.11475409836065574
0.03278688524590164
0.01639344262295082
0.0
0.0
Total atoms within 5.0 Å cutoff: 105
0.0
0.0
0.0
0.0
0.0
Total atoms within 5.0 Å cutoff: 78
0.0
0.0
0.0
0.0
0.0
Total atoms within 5.0 Å cutoff: 78
0.0
0.0
0.0
0.0
0.0
Total atoms within 5.0 Å cutoff: 87
0.0
0.0
0.0
0.0
0.09195402298850575


/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
/home/alexhernandez/miniconda3/envs/cpp-ml/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:732: UserWarning: Column atom_key is not an expected column and will be skipped.
  warn(
